# 前処理

In [1]:
!pip install --upgrade setuptools
!pip install --upgrade pip
!pip install  google-cloud-bigquery
!pip install pydata-google-auth
!pip install db-dtypes

In [7]:

import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [8]:
#BigQueryで加工済み(にじかホロのファンアートタグを含んでいた場合フラグ付し、フラグが付いたツイートのみ取得)のツイート一覧を取得

import json
from google.cloud import bigquery 

variables = open('variables.json' , 'r')
variables = json.load(variables)

client = bigquery.Client(project=variables['bq_project_name'], credentials=credentials)
query = 'SELECT * FROM '+variables['bq_table_name']+' WHERE DATE(created_at, "Japan") BETWEEN "2021-11-22" AND "2022-11-21"'
df_tmp = client.query(query).to_dataframe()
df = df_tmp

In [9]:
df.head()

,quoted_id,quoted_status_id_str,in_reply_to_status_id,quote_count,timestamp_ms,possibly_sensitive,favorited,user_id,created_at,id_str,...,quoted_status_id,in_reply_to_user_id,retweeted_id,lang,retweet_count,in_reply_to_user_id_str,text,hololive_flg,nijisanji_flg,ix
0,<NA>,None,<NA>,0,1638374815657,False,False,1364925780870922241,2021-12-01 16:06:55+00:00,1466076353267003392,...,<NA>,<NA>,<NA>,ja,0,None,イエイイエーイって感じのぺこーら描きました！\n#兎田ぺこら #ぺこらーと https://...,True,False,16632
1,<NA>,None,<NA>,0,1638375970659,False,False,70954558,2021-12-01 16:26:10+00:00,1466081197696909312,...,<NA>,<NA>,<NA>,ja,0,None,加賀美ハヤト社長、お誕生日おめでとうございます！！��������\n新たな一年も、素敵な日...,False,True,16633
2,<NA>,None,<NA>,0,1638498927660,False,False,721892949672366080,2021-12-03 02:35:27+00:00,1466596916738035713,...,<NA>,<NA>,<NA>,ja,0,None,にぇ\n#さくらみこ\n#miko_Art\n#みこちはエリート https://t.co/...,True,False,16661
3,<NA>,None,<NA>,0,1638657259660,False,False,1399838153557516292,2021-12-04 22:34:19+00:00,1467261009278558213,...,<NA>,<NA>,<NA>,ja,0,None,#ほしまちぎゃらりー ☆彡 ☆〜（ゝ。∂） https://t.co/jruhu3NkVD,True,False,16700
4,<NA>,None,<NA>,0,1638851863663,False,False,1455842042513268743,2021-12-07 04:37:43+00:00,1468077237627146240,...,<NA>,<NA>,<NA>,ja,0,None,親戚家のわんちゃん②\n#いぬいらすと #イラスト #アナログ絵 \n#トイプー #水彩画 ...,False,True,16761


In [10]:
# 各社のVTuberのファンアートタグと名前の対応表

import pandas as pd

df_tag_holo = pd.read_csv('tag_holo.csv')
df_tag_niji = pd.read_csv('tag_niji.csv')
df_tag_niji.head()

,name,tag,Twitter
0,月ノ美兎,#みとあーと,@MitoTsukino
1,勇気ちひろ,#ちーあーと,@Chihiro_yuki23
2,える,#えるの絵だよ,@Elu_World
3,樋口楓,#でろあーと,@HiguchiKaede
4,静凛,#凛Art,@ShizuRin23


In [42]:
dfs_tagged = []

for hako in ['niji', 'holo']:
    flg_name = 'nijisanji_flg' if hako == 'niji' else 'hololive_flg'
    df_tag = df_tag_niji if hako == 'niji' else df_tag_holo
    
    # df_taggedの初期化
    df_tagged = df[df[flg_name]==True][['user_id', 'created_at', 'text']]

    # タグのカラムをFalseで初期化
    for tag_name in df_tag['name'].unique():
        df_tagged[tag_name] = False

    for _, row in df_tag.iterrows():
        # 各タグが含まれているかチェックし、結果を蓄積
        df_tagged[row['name']] |= df_tagged['text'].str.contains(row['tag'])

    df_tagged.to_csv('tweets_tagged_'+hako+'.csv')
    
    dfs_tagged.append(df_tagged)

/var/folders/bd/l206qckj0ts_nk014r8kj4lm0000gn/T/ipykernel_1179/2222246832.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tagged[tag_name] = False
/var/folders/bd/l206qckj0ts_nk014r8kj4lm0000gn/T/ipykernel_1179/2222246832.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tagged[tag_name] = False
/var/folders/bd/l206qckj0ts_nk014r8kj4lm0000gn/T/ipykernel_1179/2222246832.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [44]:
dfs_tagged[0].head()

,user_id,created_at,text,月ノ美兎,勇気ちひろ,える,樋口楓,静凛,渋谷ハジメ,鈴谷アキ,...,ソフィア・ヴァレンタイン,倉持めると,佐伯イッテツ,赤城ウェン,宇佐美リト,緋八マナ,星導ショウ,叢雲カゲツ,小柳ロウ,伊波ライ
1,70954558,2021-12-01 16:26:10+00:00,加賀美ハヤト社長、お誕生日おめでとうございます！！��������\n新たな一年も、素敵な日...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1455842042513268743,2021-12-07 04:37:43+00:00,親戚家のわんちゃん②\n#いぬいらすと #イラスト #アナログ絵 \n#トイプー #水彩画 ...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,1468753168230404102,2021-12-16 13:38:15+00:00,#上弦画 \n\n描きたかったやつ〜！ https://t.co/MltIYBtnq1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13,1338056982624755713,2021-12-22 09:48:07+00:00,線画やっとできた\n#線画 #wip #葛葉 #KuzuArt #イラスト好きさんと繋が...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14,1342464767491903489,2021-12-24 13:48:01+00:00,3Dお披露目お疲れ様～！！！！\n伝えたいことをQRコードにしたのでよかったら見てね！！\n...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
import pandas as pd

dfs_tagged[0] = pd.read_csv('tweets_tagged_niji.csv')
dfs_tagged[1] = pd.read_csv('tweets_tagged_holo.csv')

In [45]:
dfs_tagged[0]['葛葉'].sum()

371

# 投稿数ランキング

In [46]:
dfs_tagged[0].iloc[:, 3:].sum().reset_index().rename(columns={0:'cnt'}).sort_values('cnt', ascending=False).head(10)

,index,cnt
107,甲斐田晴,711
16,剣持刀也,528
24,葛葉,371
81,黛灰,317
18,叶,296
96,不破湊,288
70,三枝明那,272
119,壱百満天原サロメ,216
105,長尾景,189
14,伏見ガク,171


In [47]:
dfs_tagged[1].iloc[:, 3:].sum().reset_index().rename(columns={0:'cnt'}).sort_values('cnt', ascending=False).head(10)

,index,cnt
22,宝鐘マリン,268
36,沙花叉クロヱ,227
3,星街すいせい,218
16,猫又おかゆ,195
18,兎田ぺこら,195
8,白上フブキ,193
2,さくらみこ,177
17,戌神ころね,173
10,湊あくあ,152
28,雪花ラミィ,150


In [58]:
# 先ほどアップロードされたファイルを用いてキャラクター間の共起マトリックスを作成するコード
dfs_cor = []

for i, df_tagged in enumerate(dfs_tagged):
    hako = 'niji' if i == 1 else 'holo'

    # キャラクターに関する列のみを抽出（ユーザーID、ツイート作成日、テキストを除外）
    character_data = df_tagged.drop(columns=['user_id', 'created_at', 'text'])

    # 共起マトリックスを初期化
    co_occurrence_matrix = pd.DataFrame(0, index=character_data.columns, columns=character_data.columns)

    # 各ツイートでタグ付けされたキャラクターの組み合わせをカウント
    for index, row in character_data.iterrows():
        tags = row[row == True].index
        for i in range(len(tags)):
            for j in range(i + 1, len(tags)):
                co_occurrence_matrix.at[tags[i], tags[j]] += 1
                co_occurrence_matrix.at[tags[j], tags[i]] += 1

    # 共起マトリックスをunpivotして共起回数のランキングを作成するコード

    # 共起マトリックスをunpivotして共起回数のランキングを作成
    co_occurrence_unpivoted = co_occurrence_matrix.unstack().reset_index()
    co_occurrence_unpivoted.columns = ['Character1', 'Character2', 'CoOccurrenceCount']
    co_occurrence_unpivoted = co_occurrence_unpivoted[co_occurrence_unpivoted['Character1'] != co_occurrence_unpivoted['Character2']]
    co_occurrence_ranking = co_occurrence_unpivoted.sort_values(by='CoOccurrenceCount', ascending=False)

    # Character1とCharacter2が同じ組み合わせのものをまとめる
    co_occurrence_ranking['Pair'] = co_occurrence_ranking.apply(lambda x: tuple(sorted([x['Character1'], x['Character2']])), axis=1)
    combined_ranking = co_occurrence_ranking.groupby('Pair')['CoOccurrenceCount'].sum().sort_values(ascending=False).reset_index()
    combined_ranking['CoOccurrenceCount'] = combined_ranking['CoOccurrenceCount'] / 2

    combined_ranking.to_csv('co_occurrence_ranking_'+hako+'.csv')
        
    dfs_cor.append(combined_ranking)


In [60]:
dfs_cor[0].head(10)

,Pair,CoOccurrenceCount
0,"(三枝明那, 不破湊)",91.0
1,"(叶, 葛葉)",85.0
2,"(三枝明那, 黛灰)",66.0
3,"(不破湊, 黛灰)",63.0
4,"(伏見ガク, 剣持刀也)",49.0
5,"(甲斐田晴, 長尾景)",37.0
6,"(弦月藤士郎, 長尾景)",33.0
7,"(弦月藤士郎, 甲斐田晴)",32.0
8,"(不破湊, 甲斐田晴)",32.0
9,"(剣持刀也, 加賀美ハヤト)",29.0


In [62]:
dfs_cor[1].head(10)

,Pair,CoOccurrenceCount
0,"(さくらみこ, 星街すいせい)",35.0
1,"(戌神ころね, 猫又おかゆ)",35.0
2,"(沙花叉クロヱ, 風真いろは)",24.0
3,"(獅白ぼたん, 雪花ラミィ)",24.0
4,"(沙花叉クロヱ, 鷹嶺ルイ)",22.0
5,"(博衣こより, 沙花叉クロヱ)",20.0
6,"(博衣こより, 風真いろは)",20.0
7,"(風真いろは, 鷹嶺ルイ)",18.0
8,"(桃鈴ねね, 獅白ぼたん)",18.0
9,"(兎田ぺこら, 宝鐘マリン)",18.0
